In [71]:
import re
import pandas as pd
import numpy as np
import sqlite3

In [57]:
def drop_nulls(df, col):
    return df.dropna(subset=[col])

def clean_string(s):
    """
    Removes whitespace and non-alphanumeric characters from a string.
    
    Parameters:
    s (str): The input string to clean.

    Returns:
    str: A cleaned string with only alphanumeric characters.
    """
    if isinstance(s, str):
        return re.sub(r'\W+', '', s)  # Removes non-alphanumeric characters and whitespace
    else:
        return ''  # Handle non-string cases, e.g., None or NaN


In [8]:
# Set the database path to a location with write permissions
db_path = 'race_league_results.db'

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


## set up racers table

In [56]:
racers_df.dtypes

racer_id       object
first_name     object
last_name      object
gender         object
birth_year    float64
dtype: object

In [60]:
racers_df = pd.read_csv('data/raw/FnGRacers.csv')
racers_df.drop(columns=["spouse_id"],inplace=True)
racers_df = drop_nulls(racers_df, "racer_id")
racers_df["racer_id"]=racers_df.first_name.str.lower() + racers_df.last_name.str.lower()
racers_df["racer_id"]=racers_df["racer_id"].apply(clean_string)
racers_df.head()

,racer_id,first_name,last_name,gender,birth_year
2,bradabbott,Brad,Abbott,M,1955.0
3,toriabbott,Tori,Abbott,F,NaN
4,jayadlington,Jay,Adlington,M,1975.0
5,suealexanderash,Sue,Alexander Ash,-,1961.0
6,christineallan,Christine,Allan,F,1968.0


In [61]:
racers_df.shape

(309, 5)

In [62]:
racers_df.to_sql('Racers', conn, if_exists='replace', index=False)
conn.commit()

## set up races table

In [35]:
races_df = pd.read_csv('data/raw/FnGRaces.csv')
races_df['race_date'] = pd.to_datetime(races_df['race_date'], format='%m/%d/%Y')

In [41]:
races_df.to_sql('Races', conn, if_exists='replace', index=False)
conn.commit()

In [70]:
races_df.head()

,race_id,race_date,description
0,3039,2023-02-19,2023 F&G Race#4
1,3038,2023-02-12,2023 F&G Race#3
2,3037,2023-02-05,2023 F&G Race#2
3,3036,2023-01-15,2023 F&G Race#1
4,3035,2022-12-31,2023 Time Trials


## set up race results table:

In [64]:
results_df = pd.read_csv('data/raw/FnGResults.csv')

In [65]:
results_df['racer_id'] = (
    results_df['racer_id']
    .str.split(', ')
    .apply(lambda x: x[1].lower() + x[0].lower())
    .apply(clean_string)
)

In [72]:
results_df["run1"] = results_df["_time"]
results_df["run2"] = np.nan
results_df["best_time"] = results_df["_time"]
results_df["points"] = np.nan
results_df.drop(columns=["_time"],inplace=True)

In [73]:
results_df.head()

,race_id,discipline,racer_id,run1,run2,best_time,points
0,1,SKI,jeffcox,35.82,NaN,35.82,NaN
1,1,SKI,mcleanwood,36.10,NaN,36.10,NaN
2,1,SKI,jeffparr,36.95,NaN,36.95,NaN
3,1,SKI,derekcrawford,36.97,NaN,36.97,NaN
4,1,SKI,brandonhune,37.93,NaN,37.93,NaN


In [75]:
results_df.to_sql('RaceResults', conn, if_exists='replace', index=False)
conn.commit()

## set up race team data:

In [84]:
teams_df = pd.read_csv('data/raw/24FnGStartList.csv')
teams_df = drop_nulls(teams_df, "name")
teams_df["racer_id"] = teams_df["name"].apply(clean_string)
teams_df["year"] = 2024

In [86]:
teams_df

,bib,CAT,name,tier,team,racer_id,year
0,1.0,SKI,Jennifer Hsiung,1.0,Mitch Perreault,JenniferHsiung,2024
1,2.0,SKI,Maurice Cacho,1.0,Will Carter,MauriceCacho,2024
2,4.0,SKI,Justin Rosenberg,1.0,Stephanie Coward,JustinRosenberg,2024
3,5.0,SKI,David Rosenblatt,1.0,Mike McTaggart,DavidRosenblatt,2024
4,6.0,SKI,Adam Szakacs,1.0,Don French,AdamSzakacs,2024
...,...,...,...,...,...,...,...
95,119.0,SNBD,Jenna Livingston,13.0,Stephanie Coward,JennaLivingston,2024
96,120.0,SNBD,Hannah Kilmer Choi,13.0,Will Carter,HannahKilmerChoi,2024
97,121.0,SNBD,Graham Ramshaw,13.0,Graham Ramshaw,GrahamRamshaw,2024
98,122.0,SNBD,Kevin Kilmer Choi,13.0,Adam Grossman,KevinKilmerChoi,2024


In [87]:
teams_df.to_sql('Teams', conn, if_exists='replace', index=False)
conn.commit()

## Functions to upload results

## basic views we want to display:

In [6]:
# Commit the changes and close the connection
conn.commit()
conn.close()

In [43]:
# 1. Query to view all racers
racers_query = "SELECT * FROM Racers"
racers_df = pd.read_sql_query(racers_query, conn)
print("Racers Table:")
racers_df.head()

Racers Table:


,racer_id,first_name,last_name,gender,birth_year
0,bradabbott,Brad,Abbott,M,1955.0
1,toriabbott,Tori,Abbott,F,NaN
2,jayadlington,Jay,Adlington,M,1975.0
3,suealexander ash,Sue,Alexander Ash,-,1961.0
4,christineallan,Christine,Allan,F,1968.0


In [ ]:


# Commit the changes and close the connection
conn.commit()
conn.close()
